In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [4]:
import os

In [5]:
import torch

In [6]:
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [7]:
import time
import copy
import glob

In [8]:
# data_path = '/kaggle/input/waste-classification-data/DATASET/TRAIN/'
data_path = r'data/DATASET/TRAIN'
transform = transforms.Compose(
                [
#                     transforms.Resize([256, 256]),
                    transforms.RandomResizedCrop(224),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
                ])
train_dataset = torchvision.datasets.ImageFolder(
    root=data_path,
    transform=transform
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    num_workers=8,
    shuffle=True
)

print(len(train_loader))

706


In [9]:
from torchvision.models import resnet18

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.resnet = resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(512, 1)    # replace the last full-connected layer of resnet
        self.sigmoid = nn.Sigmoid()           # use a sigmoid layer to map the logit to [0, 1]
    
    def forward(self, x):
        return self.sigmoid(self.resnet(x)).view(-1)

In [11]:
model = Net()
model

Net(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_

In [12]:
# Freeze all layers except the last two layers
for param_name, param in model.named_parameters():
    if 'fc' not in param_name:
        param.requires_grad = False

In [ ]:
# define loss function and optimizer
loss_func = nn.BCELoss() # binary cross entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # learning rate is 0.001

In [ ]:
def set_parameter_requires_grad(model, feature_extracting=True):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False  
set_parameter_requires_grad(model)

In [ ]:
model.fc = nn.Linear(512, 2)

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

In [ ]:
EPOCH = 2

model.train()
for epoch in range(EPOCH):
    for i, batch in enumerate(dataloaders):
        inputs, labels = batch
        outputs = model(inputs)            # forward propagation
        loss = loss_func(outputs, labels)  # calculate loss
        loss.backward()                    # back propagation
        optimizer.step()                   # update parameters
        optimizer.zero_grad()              # clear gradients
        print('Epoch %s | %s / %s | loss: %s' % 
              (epoch, i+1, len(dataloaders), float(loss)), end='\r')
    print()

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=25, is_train=True):
    since = time.time()
    
    acc_history = []
    loss_history = []

    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.to(torch.float32)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            # backward
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)

        print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
        if epoch_acc > best_acc:
            best_acc = epoch_acc

        acc_history.append(epoch_acc.item())
        loss_history.append(epoch_loss)
        
#         torch.save(model.state_dict(), os.path.join('/kaggle/working', '{0:0=2d}.pth'.format(epoch)))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Acc: {:4f}'.format(best_acc))
    
    return acc_history, loss_history



In [ ]:
# Here we only want to update the gradient for the classifier layer that we initialized.
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
            
optimizer = optim.Adam(params_to_update)

In [ ]:
# define loss function and optimizer
criterion = nn.BCELoss() # binary cross entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # learning rate is 0.001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Setup the loss function
# criterion = nn.CrossEntropyLoss()

# Train model
train_acc_hist, train_loss_hist = train_model(model, train_loader, criterion, optimizer, device)

In [13]:
model.resnet.conv1.weight[0][0], model.resnet.fc.weight[0][:10]

(tensor([[-0.0104, -0.0061, -0.0018,  0.0748,  0.0566,  0.0171, -0.0127],
         [ 0.0111,  0.0095, -0.1099, -0.2805, -0.2712, -0.1291,  0.0037],
         [-0.0069,  0.0591,  0.2955,  0.5872,  0.5197,  0.2563,  0.0636],
         [ 0.0305, -0.0670, -0.2984, -0.4387, -0.2709, -0.0006,  0.0576],
         [-0.0275,  0.0160,  0.0726, -0.0541, -0.3328, -0.4206, -0.2578],
         [ 0.0306,  0.0410,  0.0628,  0.2390,  0.4138,  0.3936,  0.1661],
         [-0.0137, -0.0037, -0.0241, -0.0659, -0.1507, -0.0822, -0.0058]]),
 tensor([-0.0415,  0.0165,  0.0405, -0.0170,  0.0374,  0.0065,  0.0088,  0.0247,
         -0.0099, -0.0220], grad_fn=<SliceBackward0>))

In [20]:
# define loss function and optimizer
loss_func = nn.BCELoss() # binary cross entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # learning rate is 0.001

In [27]:
EPOCH = 2

model.train()
for epoch in range(EPOCH):
    for i, batch in enumerate(train_loader):
        inputs, labels = batch
        outputs = model(inputs) 
        outputs=outputs.to(torch.float32)
        labels=labels.to(torch.float32)
        loss = loss_func(outputs, labels)  # calculate loss
        loss.backward()                    # back propagation
        optimizer.step()                   # update parameters
        optimizer.zero_grad()              # clear gradients
        print('Epoch %s | %s / %s | loss: %s' % 
              (epoch, i+1, len(train_loader), float(loss)), end='\r')
    print()

Epoch 0 | 706 / 706 | loss: 0.08934750407934189
Epoch 1 | 706 / 706 | loss: 0.334738910198211675


In [19]:
train_loader

In [28]:
# define the accuracy score
def accuracy_score(pred, y):
    return (pred.astype(int) == y.astype(int)).sum() / pred.shape[0]

In [31]:
model.eval()
preds = []
trues = []
for i, batch in enumerate(train_loader):
    inputs, labels = batch
    outputs = model(inputs)
    pred = (outputs > 0.5).detach().numpy()   # pred <= 0.5 -> cat;  pred > 0.5 -> dog 
    preds.append(pred)
    trues.append(labels.numpy())
    print('%s / %s' % (i+1, len(train_loader)), end='\r')

preds = np.concatenate(preds, axis=0)
trues = np.concatenate(trues, axis=0)
accuracy_score(preds, trues)

0.9199645468646134

In [32]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.resnet = resnet18(pretrained=True)
        #keep-the-last-fully-connected-layer-of-resnetlf.resnet = resnet18(pretrained=True)
        self.fc = nn.Linear(1000, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.resnet(x)
        out = self.sigmoid(self.fc(out))
        return out.view(-1)

In [33]:
model = Net()
model

Net(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_

In [38]:
# define loss function and optimizer
loss_func = nn.BCELoss() # binary cross entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # learning rate is 0.001

In [39]:
EPOCH = 2

model.train()
for epoch in range(EPOCH):
    for i, batch in enumerate(train_loader):
        inputs, labels = batch
        outputs = model(inputs) 
        outputs=outputs.to(torch.float32)
        labels=labels.to(torch.float32)
        loss = loss_func(outputs, labels)  # calculate loss
        loss.backward()                    # back propagation
        optimizer.step()                   # update parameters
        optimizer.zero_grad()              # clear gradients
        print('Epoch %s | %s / %s | loss: %s' % 
              (epoch, i+1, len(train_loader), float(loss)), end='\r')
    print()

Epoch 0 | 706 / 706 | loss: 0.06546648591756825
Epoch 1 | 706 / 706 | loss: 0.10812740027904512


In [42]:
model.eval()
preds = []
trues = []
for i, batch in enumerate(train_loader):
    inputs, labels = batch
    outputs = model(inputs)
    pred = (outputs > 0.5).detach().numpy()   # pred <= 0.5 -> cat;  pred > 0.5 -> dog 
    preds.append(pred)
    trues.append(labels.numpy())
    print('%s / %s' % (i+1, len(train_loader)), end='\r')

preds = np.concatenate(preds, axis=0)
trues = np.concatenate(trues, axis=0)
accuracy_score(preds, trues)

0.8229115887436295

In [41]:
model.eval()
preds = []
trues = []
for i, batch in enumerate(train_loader):
    inputs, labels = batch
    outputs = model(inputs)
    pred = (outputs > 0.5).detach().numpy()   # pred <= 0.5 -> cat;  pred > 0.5 -> dog 
    preds.append(pred)
    trues.append(labels.numpy())
    print('%s / %s' % (i+1, len(train_loader)), end='\r')

preds = np.concatenate(preds, axis=0)
trues = np.concatenate(trues, axis=0)
accuracy_score(preds, trues)

1.0